In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from fairdata import FairData

In [2]:
df_raw_1 = pd.read_excel('data/Fintech/Fintech-fairnessJun2020.xlsx', sheet_name='Approved&Default')
df_raw_2 = pd.read_excel('data/Fintech/Fintech-fairnessJun2020.xlsx', sheet_name='Rejected')
df_raw_1['approved_dum'] = 1
df_raw_2['approved_dum'] = 0
df_tmp = pd.concat([
    df_raw_1.drop(['loan_transferred_date', 'def_flag'], axis=1),
    df_raw_2.rename(columns={'loan_request_initial_id': 'loan_request_id'}),
]).rename(columns={
    'noofconnections': 'connections',
    'noofapps': 'apps',
    'noofsms': 'sms',
    'noofcontacts': 'contacts',
}).astype({
    'customer_id': 'Int64',
    'loan_request_id': 'Int64',
    'age': 'float',
    'connections': 'float',
    'apps': 'float',
    'sms': 'float',
    'contacts': 'float',
    'approved_dum': 'Int64',
})
df_raw_3 = pd.read_csv('data/Fintech/Cashe_information.csv').rename(columns={
    'AGE': 'age',
}).astype({
    'customer_id': 'Int64',
    'loan_request_id': 'Int64',
    'age': 'float',
    'connections': 'float',
    'apps': 'float',
    'sms': 'float',
    'contacts': 'float',
    'approved_dum': 'Int64',
})
df_raw = pd.merge(
    df_tmp.dropna(),
    df_raw_3.loc[:, ['loan_request_id', 'salary', 'loan_amount', 'CIBIL']].dropna(), 
    how='left', on='loan_request_id').dropna()
df_raw

,customer_id,loan_request_id,age,connections,apps,sms,contacts,gender,approved_dum,salary,loan_amount,CIBIL
0,150,218,32.0740,811.0,10.0,584.0,1820.0,m,1,40000.0,5000.0,707.0
1,150,97964,32.0740,811.0,10.0,584.0,1820.0,m,1,40000.0,40000.0,707.0
2,150,66,32.0740,811.0,10.0,584.0,1820.0,m,1,40000.0,5000.0,707.0
3,150,2312,32.0740,811.0,10.0,584.0,1820.0,m,1,40000.0,5000.0,707.0
4,150,3754,32.0740,811.0,10.0,584.0,1820.0,m,1,40000.0,5000.0,707.0
...,...,...,...,...,...,...,...,...,...,...,...,...
261924,1600000,282451,18.3178,1.0,24.0,121.0,171.0,m,0,115000.0,200000.0,747.0
261958,1600000,283249,30.0904,736.0,57.0,1638.0,862.0,m,0,50251.0,101000.0,663.0
261970,1600000,283473,18.6904,196.0,37.0,1328.0,203.0,m,0,68328.0,137000.0,745.0
261987,1600000,283850,27.9096,9.0,48.0,237.0,878.0,m,0,32012.0,64000.0,716.0


In [4]:
del df_raw_1, df_raw_2, df_raw_3, df_tmp

NameError: name 'df_raw_1' is not defined

In [5]:
df_gender = df_raw[~df_raw.gender.isna()]
df_gender['gender_dum'] = 1
df_gender.loc[df_gender.gender == 'f', 'gender_dum'] = 0
df_gender = df_gender.drop('gender', axis=1).astype({'approved_dum': 'int64'})
df_g = df_gender.drop(['customer_id', 'loan_request_id'], axis=1)
log_vars = ['loan_amount', 'salary', 'connections',  'apps', 'sms', 'contacts']
for c in log_vars:
    df_g[c] = np.log(df_g[c] + 1)
norm_vars = log_vars + ['age', 'CIBIL']
scaler = StandardScaler().fit(df_g[norm_vars])
df_g[norm_vars] = scaler.transform(df_g[norm_vars])

In [6]:
df_age = df_gender.copy()
df_age['age_dum'] = 1
df_age.loc[df_age.age < 28, 'age_dum'] = 0
df_a = df_age.drop(['customer_id', 'loan_request_id', 'age', 'CIBIL', 'loan_amount'], axis=1)
log_vars = ['salary', 'connections',  'apps', 'sms', 'contacts']
for c in log_vars:
    df_a[c] = np.log(df_a[c] + 1)
norm_vars = log_vars
scaler = StandardScaler().fit(df_a[norm_vars])
df_a[norm_vars] = scaler.transform(df_a[norm_vars])

In [7]:
df_b = df_a.copy()
df_b['dum'] = 0
df_b.loc[(df_b.age_dum == 0) & (df_b.gender_dum == 1), 'dum'] = 1
df_b.loc[(df_b.age_dum == 1) & (df_b.gender_dum == 0), 'dum'] = 2
df_b.loc[(df_b.age_dum == 1) & (df_b.gender_dum == 1), 'dum'] = 3
df_b

,connections,apps,sms,contacts,approved_dum,salary,gender_dum,age_dum,dum
0,0.742061,-2.373794,-0.045504,0.890059,1,0.432050,1,1,3
1,0.742061,-2.373794,-0.045504,0.890059,1,0.432050,1,1,3
2,0.742061,-2.373794,-0.045504,0.890059,1,0.432050,1,1,3
3,0.742061,-2.373794,-0.045504,0.890059,1,0.432050,1,1,3
4,0.742061,-2.373794,-0.045504,0.890059,1,0.432050,1,1,3
...,...,...,...,...,...,...,...,...,...
261924,-2.702040,-1.024403,-0.808190,-0.729581,0,2.584841,1,0,1
261958,0.686491,0.358826,0.455738,0.377513,0,0.897138,1,1,3
261970,-0.070057,-0.336196,0.353732,-0.612465,0,1.523554,1,0,1
261987,-1.779172,0.081672,-0.483064,0.390122,0,-0.022064,1,0,1


In [8]:
np.random.seed(1)

y = df_b.approved_dum.values.reshape(-1, 1)
s = df_b.dum.values.reshape(-1, 1)
a = df_b.drop(['approved_dum', 'age_dum', 'gender_dum', 'dum'], axis=1).values

n_test = 10000

shuffle_idx = np.arange(y.shape[0])
np.random.shuffle(shuffle_idx)
y_train, y_test = y[shuffle_idx[n_test:]], y[shuffle_idx[:n_test]]
s_train, s_test, s_eval = s[shuffle_idx[n_test:]], s[shuffle_idx[:n_test]], np.array([0, 1] * n_test).reshape(-1, 1)
a_train, a_test, a_eval = a[shuffle_idx[n_test:]], a[shuffle_idx[:n_test]], np.repeat(a[shuffle_idx[:n_test]], 2, axis=0)

In [11]:
df_b_fair_o = FairData(s_train, a_train, y_train, preprocess_method='o')
eval_o = df_b_fair_o.evaluate(a_test, s_test, y_test, ['cf', 'mae'])
pd.DataFrame(eval_o, index=['cf', 'mae'], columns=['ML', 'FTU', 'FL', 'AA', 'FLAP-1', 'FLAP-2'])

,ML,FTU,FL,AA,FLAP-1,FLAP-2
cf,0.039234,0.013032,0.001115,0.001123,0.001117,0.001115
mae,0.124886,0.126101,0.125834,0.126567,0.125835,0.125834


In [12]:
df_b_fair_m = FairData(s_train, a_train, y_train, preprocess_method='m')
eval_m = df_b_fair_m.evaluate(a_test, s_test, y_test, ['cf', 'mae'])
pd.DataFrame(eval_m, index=['cf', 'mae'], columns=['ML', 'FTU', 'FL', 'AA', 'FLAP-1', 'FLAP-2'])

,ML,FTU,FL,AA,FLAP-1,FLAP-2
cf,0.039234,0.013032,0.001115,0.001123,0.000766,0.000745
mae,0.124886,0.126101,0.125834,0.126567,0.125755,0.125772


In [30]:
df_b_fair_m.cit_parametric(summary=True)

<F test: F=array([[25127.24149695]]), p=0.0, df_denom=1.94e+05, df_num=3>


0.0

In [59]:
p = df_b_fair_m.f_2(s_train, a_train, 'm')
y_hat = np.random.binomial(1, 0.95, len(y_train)).reshape(-1, 1)
df_b_processed = FairData(s_train, a_train, y_hat, preprocess_method='m')
df_b_processed.cit_parametric(summary=True)

<F test: F=array([[25759.31690959]]), p=0.0, df_denom=1.94e+05, df_num=3>


0.0